In [32]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

url = 'https://www.wunderground.com/history/daily/sd/khartoum/HSSS/date/2019-03-12'

# Replace the path below with the actual path to your chromedriver executable
chrome_driver_path = '/Users/samdvorin/Desktop/code/542/weather-prediction/chromedriver'

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')  # Run Chrome in headless mode (without a graphical user interface)
chrome_options.add_argument('--disable-gpu')  # Disable GPU acceleration in headless mode

# Use 'executable_path' as a service argument, not as an option
chrome_service = webdriver.chrome.service.Service(chrome_driver_path)
driver = webdriver.Chrome(service=chrome_service, options=chrome_options)

driver.get(url)

tables = WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "table")))

for table in tables:
    newTable = pd.read_html(table.get_attribute('outerHTML'))
    if newTable:
        print(newTable[0].fillna(''))

driver.quit()


NoSuchDriverException: Message: Unable to locate or obtain driver for chrome; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors/driver_location
